In [3]:
import numpy as np
import pickle
import random

In [4]:
filenames = pickle.load(open('data/filenames-caltech101.pickle', 'rb'))
feature_list = pickle.load(open('data/features-caltech101-mobilenet.pickle',
                                'rb'))

In [5]:
num_images = len(filenames)
num_features_per_image = len(feature_list[0])

PCA

In [7]:
num_feature_dimensions = 100
num_feature_dimensions = min(num_images, num_feature_dimensions,
                             len(feature_list[0]))

In [8]:
from sklearn.decomposition import PCA

In [9]:
pca = PCA(n_components=num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list)
feature_list_compressed = feature_list_compressed.tolist()

In [1]:
import falconn

In [10]:
# Setup different parameters for Falonn
parameters = falconn.LSHConstructionParameters()
num_tables = 1
parameters.l = num_tables
parameters.dimension = num_feature_dimensions
parameters.distance_function = falconn.DistanceFunction.EuclideanSquared
parameters.lsh_family = falconn.LSHFamily.CrossPolytope
parameters.num_rotations = 1
parameters.num_setup_threads = 1
parameters.storage_hash_table = falconn.StorageHashTable.BitPackedFlatHashTable

# Train the Falconn model
falconn.compute_number_of_hash_functions(16, parameters)

falconn On One Image 

In [13]:
random_image_index = random.randint(0, num_images)

In [14]:
dataset = np.array(feature_list_compressed)
a = np.random.randn(8677, 100)
a /= np.linalg.norm(a, axis=1).reshape(-1, 1)
dataset = a

index = falconn.LSHIndex(parameters)
%time index.setup(dataset)

query_object = index.construct_query_object()
num_probes = 1
query_object.set_num_probes(num_probes)

searchQuery = np.array(feature_list_compressed[random_image_index])
searchQuery = a[0]
%timeit query_object.find_k_nearest_neighbors(searchQuery, 5)

CPU times: user 30.5 ms, sys: 743 µs, total: 31.3 ms
Wall time: 18.1 ms
8.33 µs ± 2.11 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


On total images

In [15]:
searchQuery = np.array(feature_list_compressed)
searchQuery = a[0]
%timeit query_object.find_k_nearest_neighbors(searchQuery, 5)

7.79 µs ± 1.42 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
